In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score


In [2]:
file_path = r"D:\MINE\大数据分析综合实验\大数据2102班_彭弋桐_基于小米su7评论的情感分析和数据分析\数据\情感赋值.csv"
df = pd.read_csv(file_path)
df

,用户名,内容,点赞量,日期,amend_weight_sum,length
0,SupremeWow,小米su7强势破圈太火爆了，雷总得抓紧提高产能啊,3025,2024-03-30 12:22:53,2.0,24
1,及米侠,感觉这次黑稿不太多，看来车圈还是更干净一些！,1725,2024-03-30 12:23:26,1.0,22
2,小麦Elite,小米手机1代回忆～,1020,2024-03-30 12:22:38,NaN,9
3,米12U,雷总辛苦了 感谢您为大家做了一辆Dream car,660,2024-03-30 12:22:39,1.0,25
4,Spider_boy666,雷总加电,650,2024-03-30 12:23:00,NaN,4
...,...,...,...,...,...,...
3411,数毛闲聊站,让车为人服务越来越好,0,2024-03-30 12:22:33,NaN,10
3412,我在梦里见过你821,红米还出汽车吗？,0,2024-03-30 12:22:15,1.0,8
3413,夏天的尾巴_Z,小米fold3为啥造的？,0,2024-03-30 12:22:13,NaN,12
3414,My咪西mi,suv,0,2024-03-30 12:22:12,NaN,3


In [3]:
# 删除 '内容' 列中包含空值的所有行
df = df.dropna()

# 删除重复
df = df.drop_duplicates()
df.shape

(1669, 6)

In [8]:
# 手动将点赞量离散化为分类变量
def classify_likes(likes):
    if likes <= 500:
        return '低'
    elif likes <= 1500:
        return '中'
    else:
        return '高'

df['点赞量分类'] = df['点赞量'].apply(classify_likes)

# 手动对标签进行数值化
df.loc[df['点赞量分类'] == '低', '点赞量分类_数值'] = 1
df.loc[df['点赞量分类'] == '中', '点赞量分类_数值'] = 2
df.loc[df['点赞量分类'] == '高', '点赞量分类_数值'] = 3

# 将日期列转换为日期时间类型
df['日期'] = pd.to_datetime(df['日期'])

# 计算与参考日期（2024/3/30 12:22）的分钟差
reference_date = pd.Timestamp('2024-03-30 12:22')
df['分钟差'] = (df['日期'] - reference_date) / pd.Timedelta(minutes=1) 
# 将时间差转换为分钟

df.head()


,用户名,内容,点赞量,日期,amend_weight_sum,length,点赞量分类,点赞量分类_数值,分钟差
0,SupremeWow,小米su7强势破圈太火爆了，雷总得抓紧提高产能啊,3025,2024-03-30 12:22:53,2.0,24,高,3.0,0.883333
1,及米侠,感觉这次黑稿不太多，看来车圈还是更干净一些！,1725,2024-03-30 12:23:26,1.0,22,高,3.0,1.433333
3,米12U,雷总辛苦了 感谢您为大家做了一辆Dream car,660,2024-03-30 12:22:39,1.0,25,中,2.0,0.650000
5,柴柴chaial啊,我今天看了一早上雷总的发布会，尤其在汽车发布会最后那个鞠躬，让人泪目,469,2024-03-30 12:25:07,1.0,34,低,1.0,3.116667
7,钟雨飞,年轻人特别喜欢SU7，定位太成功了！！！,297,2024-03-30 12:24:11,2.0,20,低,1.0,2.183333


In [13]:
 #特征和标签
X = df[['amend_weight_sum', 'length', '分钟差']]
y = df['点赞量分类_数值']


# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# 模型训练
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model_lr.fit(X_train, y_train) 


# 预测
y_pred = model_lr.predict(X_test)
# print(y_pred)


# 评估模型
accuracy_lr = accuracy_score(y_test, y_pred)
print(accuracy_lr)

0.9960079840319361


In [19]:
coefficients = model_lr.coef_
coefficients

array([[ 0.12118787, -0.03923192,  0.87654228],
       [-0.37427705,  0.03405033, -0.5480372 ],
       [ 0.25308918,  0.00518159, -0.32850509]])

随机森林

In [16]:

# 手动将点赞量离散化为分类变量
def classify_likes(likes):
    if likes <= 500:
        return '低'
    elif likes <= 1500:
        return '中'
    else:
        return '高'

df['点赞量分类'] = df['点赞量'].apply(classify_likes)

# # 手动对标签进行数值化
df.loc[df['点赞量分类'] == '低', '点赞量分类_数值'] = 1
df.loc[df['点赞量分类'] == '中', '点赞量分类_数值'] = 2
df.loc[df['点赞量分类'] == '高', '点赞量分类_数值'] = 3

# 将日期列转换为日期时间类型
df['日期'] = pd.to_datetime(df['日期'])

# 计算与参考日期（2024/3/30 12:22）的分钟差
reference_date = pd.Timestamp('2024-03-30 12:22')
df['分钟差'] = (df['日期'] - reference_date) / pd.Timedelta(minutes=1)  # 将时间差转换为分钟

 #特征和标签
X = df[['amend_weight_sum', 'length', '分钟差']]
y = df['点赞量分类_数值']
# y = df['点赞量分类']

# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

# 模型训练
from sklearn.ensemble import RandomForestClassifier #引/入分类决策树模型
from sklearn.metrics import accuracy_score
model_rf = RandomForestClassifier(max_depth=3,n_estimators =10, min_samples_leaf=10,random_state=1)
model_rf.fit(X_train,y_train)


# 预测
y_pred = model_rf.predict(X_test)

# 评估模型
accuracy_rf = accuracy_score(y_test, y_pred)
accuracy_rf

1.0

In [7]:

features = X.columns
importances2 = model_rf.feature_importances_

importances_rf = pd.DataFrame()
importances_rf['特征名称'] = features
importances_rf['特征重要性'] = importances2
importances_rf.sort_values('特征重要性',ascending=False)
importances_rf

,特征名称,特征重要性
0,amend_weight_sum,0.064054
1,length,0.254842
2,分钟差,0.681104
